In [7]:

from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter , Language
from langchain_ollama.chat_models import ChatOllama
from langchain.memory import ConversationSummaryMemory
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

repo_path = "C:\VScodeMaster\GeminiAPI"

In [8]:


loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       show_progress=True,
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
                                        )

In [9]:
documents = loader.load()
chunks = RecursiveCharacterTextSplitter.from_language(Language.PYTHON,chunk_size = 2000,
                                                             chunk_overlap = 200).split_documents(documents=documents)

  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
embeddings = OllamaEmbeddings(model="qwen2.5-coder:7b",)
# LLM 
llm = ChatOllama(model='qwen2.5-coder:7b',keep_alive='2h',num_predict = 90000)

memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [11]:
vectordb = Chroma.from_documents(chunks, embedding=embeddings) # , persist_directory='./db' for the persistent storage
vectordb._persist_directory

'./chroma'

In [12]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [13]:
PROMPT = """
Youre a software engineer and you have to explain the given code based on the context from retrieved documents.
Explain every thing in a detailed manner like youre having a Knowledge Transfer session with a colleague.
Things you need to do give explanation of the code start from classes and their methods and working and so on. DO NOT SUMMARIZE.
explain each function in detail and how it works and what it does. if asked only a function explain that function only with reference to the file it is used.
DO NOT SUMMARIZE.
context: {context}
Question:{input}
Answer:
"""
prompt = ChatPromptTemplate.from_template(PROMPT)

stuff_chain = create_stuff_documents_chain(llm=llm,prompt=prompt, output_parser=StrOutputParser())

In [14]:
retrieval_chain = create_retrieval_chain(retriever=vectordb.as_retriever(),combine_docs_chain=stuff_chain)

In [16]:
for i in retrieval_chain.stream({"input": """explain the code base in detail please """}):
    try:
        if i['answer']:
            print(i['answer'], end="",flush=True)
    except:
        pass

Certainly! Let's break down the provided codebase step by step. This code is a FastAPI application that handles chat-related operations, including creating, retrieving, and deleting chats.

### **1. FastAPI Setup**

First, we have the FastAPI setup with some imports:

```python
from fastapi import FastAPI
import models
from dotenv import load_dotenv
from database import sql_engine
from repo import Userrepo
from routers import User, Chat, authentication
```

- **FastAPI**: The main framework for building API.
- **models**: This module contains SQLAlchemy models that define the structure of the database tables.
- **dotenv**: A Python library used to load environment variables from a `.env` file.
- **sql_engine**: An instance of an SQL engine created using SQLAlchemy, which is used to connect to the database.
- **Userrepo**: Repository for handling user-related operations (likely in `repo/user.py`).
- **authentication**: Router for authentication-related endpoints (likely in `routers/auth